In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import heapq
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class Task:
    def __init__(self, process, arrival, burst, priority):
        self.process = process
        self.arrival = arrival
        self.burst = burst
        self.priority = priority
        self.start_time = None
        self.end_time = None
        self.waiting_time = None

    def __lt__(self, other):
        return (self.burst < other.burst) or (self.burst == other.burst and self.priority > other.priority)

class SchedulerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("⚡ CPU scheduling algorithm that minimizes energy consumption")
        self.tasks = []
        self.setup_ui()

    def setup_ui(self):
        self.root.configure(bg="#1e1e2f")
        header = tk.Label(self.root, text="⚡ CPU scheduling algorithm that minimizes energy consumption",
                          font=("Segoe UI", 18, "bold"), bg="#1e1e2f", fg="#88c0d0")
        header.pack(pady=15)

        input_frame = tk.Frame(self.root, bg="#2e3440")
        input_frame.pack(pady=10)

        labels = ["Process", "Arrival Time", "Burst Time", "Priority (1-5)"]
        for idx, text in enumerate(labels):
            tk.Label(input_frame, text=text, bg="#2e3440", fg="#d8dee9", font=("Segoe UI", 10)).grid(row=0, column=idx, padx=10, pady=5)

        self.entries = []
        for i in range(5):
            row = []
            for j in range(4):
                entry = tk.Entry(input_frame, width=12, font=("Segoe UI", 10))
                entry.grid(row=i+1, column=j, padx=5, pady=5)
                row.append(entry)
            self.entries.append(row)

        button_frame = tk.Frame(self.root, bg="#1e1e2f")
        button_frame.pack(pady=10)

        tk.Button(button_frame, text="Run Scheduler", command=self.schedule,
                  bg="#a3be8c", fg="black", font=("Segoe UI", 10, "bold"), width=15).pack(side="left", padx=10)

        tk.Button(button_frame, text="Exit", command=self.root.quit,
                  bg="#bf616a", fg="white", font=("Segoe UI", 10, "bold"), width=10).pack(side="left", padx=10)

        self.figure = Figure(figsize=(8, 3), dpi=100)
        self.ax = self.figure.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.root)
        self.canvas.get_tk_widget().pack(pady=5)

        style = ttk.Style()
        style.theme_use("default")
        style.configure("Treeview", background="#3b4252", foreground="white", fieldbackground="#3b4252", font=("Segoe UI", 9))
        style.configure("Treeview.Heading", background="#434c5e", foreground="#eceff4", font=("Segoe UI", 10, "bold"))

        self.result_table = ttk.Treeview(self.root,
                                         columns=("Process", "Arrival", "Burst", "Priority", "Start", "End", "Wait", "Energy"),
                                         show="headings")
        for col in self.result_table["columns"]:
            self.result_table.heading(col, text=col)
            self.result_table.column(col, anchor="center", width=90)
        self.result_table.pack(fill="both", expand=True, padx=20, pady=10)

    def schedule(self):
        self.tasks.clear()
        self.result_table.delete(*self.result_table.get_children())
        try:
            for row in self.entries:
                if all(cell.get() for cell in row):
                    process = int(row[0].get())
                    arrival = int(row[1].get())
                    burst = int(row[2].get())
                    priority = int(row[3].get())
                    self.tasks.append(Task(process, arrival, burst, priority))

            self.tasks.sort(key=lambda t: t.arrival)
            time = 0
            ready_queue = []
            energy_used = 0
            i = 0
            gantt_data = []

            while i < len(self.tasks) or ready_queue:
                while i < len(self.tasks) and self.tasks[i].arrival <= time:
                    heapq.heappush(ready_queue, self.tasks[i])
                    i += 1
                if not ready_queue:
                    time += 1
                    continue
                current = heapq.heappop(ready_queue)
                current.start_time = time
                time += current.burst
                current.end_time = time
                current.waiting_time = current.start_time - current.arrival
                task_energy = current.burst * (1 / current.priority)
                energy_used += task_energy
                gantt_data.append((current.process, current.start_time, current.end_time))
                self.result_table.insert("", "end", values=(current.process, current.arrival, current.burst,
                                                            current.priority, current.start_time,
                                                            current.end_time, current.waiting_time,
                                                            round(task_energy, 2)))

            self.ax.clear()
            for i, (process, start, end) in enumerate(gantt_data):
                self.ax.broken_barh([(start, end - start)], (i - 0.4, 0.8), facecolors='#88c0d0')
                self.ax.text((start + end) / 2, i, f"P{process}", ha='center', va='center', color='black', fontsize=9)
            self.ax.set_title("Gantt Chart - Task Execution Timeline", fontsize=10)
            self.ax.set_xlabel("Time")
            self.ax.set_yticks([])
            self.canvas.draw()

            messagebox.showinfo("Success", f"Scheduling completed!\nTotal Energy Used: {round(energy_used, 2)}")

        except Exception as e:
            messagebox.showerror("Error", f"Invalid input or processing error:\n{e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = SchedulerApp(root)
    root.mainloop()
